<a href="https://colab.research.google.com/github/sileod/pragmeval/blob/master/run_pragmeval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.system('pip install transformers datasets pygrove xpflow funcy')
import pygrove
from xpflow import Xp
from datasets import get_dataset_config_names

In [ ]:
metric=load_metric("glue","mnli")

def preprocess_function(examples):
    if 'sentence1' in examples:
        return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding="max_length",max_length=256)
    else:
        return tokenizer(examples['sentence'], truncation=True, padding="max_length",max_length=256)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)
"""
Replace model_name by your model and adjust gradient_accumulation_steps to suit your gpu size
"""
class evaluate(Xp):
    model_name='roberta-base' # Your model
    dataset_name=get_dataset_config_names('pragmeval')
    num_train_epochs=3
    batch_size=16
    learning_rate=2e-5
    gradient_accumulation_steps=1
    warmup_ratio=0.06
    seed=0


for xp in evaluate():
    tokenizer=AutoTokenizer.from_pretrained(xp.model_name)
    dataset=load_dataset('pragmeval',xp.dataset_name)
    encoded_dataset = dataset.map(preprocess_function, batched=True)
    num_labels=dataset['train'].features['label'].num_classes
    model=AutoModelForSequenceClassification.from_pretrained(xp.model_name,num_labels=num_labels)

    args = TrainingArguments(
                xp.dataset_name,
                save_strategy="no",
                save_steps=1e9,
                evaluation_strategy = "epoch",
                per_device_train_batch_size=xp.batch_size//xp.gradient_accumulation_steps,
                per_device_eval_batch_size=xp.batch_size//xp.gradient_accumulation_steps,
                **fc.project(dict(xp), dir(TrainingArguments)),
                fp16_opt_level="O1",
            )
    trainer = Trainer(
        model,
        args,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    print(xp, trainer.predict(encoded_datasets["test"]).metrics) # log your results here

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.4",
  "type_vocab_size": 1,
  "use_cach

  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/pragmeval/verifiability/1.0.0/5f81e0f3de285b149e9a5b1748a67c6973c88bab973c7516ac0f85bc337d98ba/cache-0dc9f353fcb012a6.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/pragmeval/verifiability/1.0.0/5f81e0f3de285b149e9a5b1748a67c6973c88bab973c7516ac0f85bc337d98ba/cache-9ccde1fdf0f7e954.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/pragmeval/verifiability/1.0.0/5f81e0f3de285b149e9a5b1748a67c6973c88bab973c7516ac0f85bc337d98ba/cache-a9a97e5d53182ce7.arrow
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob":

Epoch,Training Loss,Validation Loss
